In [1]:
suppressPackageStartupMessages(library(tidyverse))
library(lubridate)
library(glue)
# library(bupaR)
# library(processanimateR)
library(survival)
library(ggplot2)
library(ggfortify)
library(patchwork)
library(ggsurvfit)
library(survminer)
library(gtsummary)
suppressPackageStartupMessages(library(flextable))
suppressPackageStartupMessages(library(Greg))

Loading required package: ggpubr


Attaching package: ‘survminer’


The following object is masked from ‘package:survival’:

    myeloma




In [2]:
est_ci_fn <- function(df) {
    as.character(df %>% reframe(glue::glue("{round(estimate, 2)} ({round(conf.low, 2)}--{round(conf.high,2)})")) %>% pull())
}

regression_summary <- function(grouping_number, ed_adn_type = "non_avoid") {

    basic_avoid_df <- readRDS(glue::glue("output/grouping{grouping_number}_cox_basic_non_avoid.rds"))
    adj_avoid_df <- readRDS(glue::glue("output/grouping{grouping_number}_cox_adj_non_avoid.rds"))
    basic_all_df <- readRDS(glue::glue("output/grouping{grouping_number}_cox_basic_all.rds"))
    adj_all_df <- readRDS(glue::glue("output/grouping{grouping_number}_cox_adj_all.rds"))
    summary_df <- readRDS(glue::glue("output/grouping{grouping_number}_summary_df.rds")) %>%
        count(strata)
    
    # mutate(across(telex:players, ~ format(., big.mark = ",", scientific = F)))
    un_hr_avoid = est_ci_fn(basic_avoid_df[[2]])
    adj_hr_avoid = est_ci_fn(adj_avoid_df[[2]][1,])
    un_hr_all = est_ci_fn(basic_all_df[[2]])
    adj_hr_all = est_ci_fn(adj_all_df[[2]][1,])

    #print(un_hr_avoid)
    
    result <- tibble(
        grouping = glue::glue("Grouping {grouping_number}"),
        abn = summary_df$n[summary_df$strata == "ABN"],
        iuc = summary_df$n[summary_df$strata == "IUC"],
        un_hr_avoid = un_hr_avoid,
        adj_hr_avoid = adj_hr_avoid,
        un_hr_all = un_hr_all,
        adj_hr_all = adj_hr_all
    )

    result
    
    
}

In [3]:
cox_summary_tbl <- dplyr::bind_rows(
    regression_summary(1),
    regression_summary(2),
    regression_summary(3)
)

In [4]:
cox_summary_tbl

grouping,abn,iuc,un_hr_avoid,adj_hr_avoid,un_hr_all,adj_hr_all
<glue>,<int>,<int>,<chr>,<chr>,<chr>,<chr>
Grouping 1,1938,172137,1.3 (1.15--1.47),1.37 (1.21--1.55),1.17 (1.05--1.3),1.35 (1.22--1.5)
Grouping 2,19057,168612,3.35 (3.15--3.56),1.42 (1.33--1.5),3.21 (3.04--3.39),1.43 (1.35--1.51)
Grouping 3,20995,168623,2.92 (2.76--3.08),1.42 (1.34--1.5),2.76 (2.63--2.9),1.43 (1.36--1.5)


In [5]:
cst <- cox_summary_tbl %>%
    rename(
        Grouping = grouping,
        `Abandoned calls (N)` = abn,
        `Triaged 111 calls (N)` = iuc,
        `Unadjusted HR (95% CI)` = un_hr_avoid,
        `Adjusted HR (95% CI)` = adj_hr_avoid,
        `Unadjusted HR (95%CI)` = un_hr_all,
        `Adjusted HR (95%CI)` = adj_hr_all
    ) %>%
    as_flextable() %>%
    add_header_row(
      values = c("", "Non-avoidable ED attendances", "All ED attendances"),
      colwidths = c(3, 2, 2)
    )

cst

a flextable object.
col_keys: `Grouping`, `Abandoned calls (N)`, `Triaged 111 calls (N)`, `Unadjusted HR (95% CI)`, `Adjusted HR (95% CI)`, `Unadjusted HR (95%CI)`, `Adjusted HR (95%CI)` 
header has 3 row(s) 
body has 3 row(s) 
original dataset sample: 
    Grouping Abandoned calls (N) Triaged 111 calls (N) Unadjusted HR (95% CI)
1 Grouping 1                1938                172137       1.3 (1.15--1.47)
2 Grouping 2               19057                168612      3.35 (3.15--3.56)
3 Grouping 3               20995                168623      2.92 (2.76--3.08)
  Adjusted HR (95% CI) Unadjusted HR (95%CI) Adjusted HR (95%CI)
1    1.37 (1.21--1.55)      1.17 (1.05--1.3)    1.35 (1.22--1.5)
2     1.42 (1.33--1.5)     3.21 (3.04--3.39)   1.43 (1.35--1.51)
3     1.42 (1.34--1.5)      2.76 (2.63--2.9)    1.43 (1.36--1.5)

In [6]:
saveRDS(cst, 'output/cox_regression_flex_table.rds')

In [7]:
cst %>%
    save_as_docx(path = glue::glue("output/cox_regression_table.docx"))